# Working with Snowflake

In [1]:
## Packages
import swat
import sys
import os
import pandas as pd
import numpy as np
import json


## Options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

## My custom connection package for CAS
try:
    from casauth import CASAuth
    print('Imported personal custom CAS auth package')
except:
    print('casauth package not available')

    
## View versions of packages
print(f'Python version:{sys.version.split("|")[0]}')
print(f'swat version:{swat.__version__}')
print(f'pandas version:{pd.__version__}')
print(f'numpy version:{np.__version__}')

Imported personal custom CAS auth package
Python version:3.11.5 
swat version:1.13.3
pandas version:2.0.3
numpy version:1.24.3


In [2]:
swat?

Type:        module
String form: <module 'swat' from 'C:\\Users\\pestyl\\AppData\\Local\\anaconda3\\Lib\\site-packages\\swat\\__init__.py'>
File:        c:\users\pestyl\appdata\local\anaconda3\lib\site-packages\swat\__init__.py
Docstring:  
SAS Scripting Wrapper for Analytics Transfer (SWAT)

This package allows you to connect to a SAS CAS host and call actions.
The responses and results are returned as Python objects.

Create a connection
-------------------

>>> s = CAS('myhost.com', 12345, 'username', 'password')

Load a data table
-----------------

>>> tbl = s.read_path('datasources/cars_single.sashdat')

Load an action set
------------------

>>> s.loadactionset(actionset='simple')

Get help for an action set
--------------------------

>>> help(s.simple)  # or s.simple? in IPython

Get help for an action
----------------------

>>> help(s.summary)  # or s.summary? in IPython

Call an action from the library
-------------------------------

>>> result = tbl.summary()
>>> print(re

## Connect to CAS
My personal CAS connection information.

In [2]:
###################################################
## My Personal connection using a custom package ##
###################################################
path = os.getenv('CAS_CREDENTIALS')
pem_file = os.getenv('CAS_CLIENT_SSL_CA_LIST')

conn = CASAuth(path, ssl_ca_list = pem_file)

################################
## General connection syntax  ##
################################
# conn = swat.CAS(host, port, username, password)

##########################################
## SAS Viya for Learners 3.5 connection ##
##########################################
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue, port=portValue, password=passwordToken)

CAS Connection created


Current version of SAS Viya.

In [14]:
conn.about()['About']['Viya Version']

NOTE: Grid node action status report: 5 nodes, 26 total actions executed.


'Stable 2023.10'

## Connect to Snowflake
Connect to my trial Snowflake environment. I'm using a simple username and password to connect to Snowflake and have stored all of my information in a JSON file. **Please follow any company guidelines to connect to your production Snowflake environment.**

My JSON file template:

{

    "srctype" : "snowflake",
    "server"  : "<your account>.snowflakecomputing.com",
    "userName": "user-name",
    "password": "password"
}

For more information about connecting to a Snowflake database:
- [SAS Viya Best practices with Snowflake Data](https://video.sas.com/detail/video/6312274491112/sas-viya-best-practices-with-snowflake-data)
- [Documentation - Snowflake Data Connector](https://go.documentation.sas.com/doc/en/pgmsascdc/default/casref/p183rli8obtde3n10y9bzbrpwnsh.htm)

In [ ]:
## Get my Snowflake connection information
snow_creds = json.load(open(os.getenv('CAS_CREDENTIALS') + '\snowflake_creds.json'))

## Create a caslib to Snowflake using my connection information
cr = conn.addcaslib(name = 'my_snow_db',
                    datasource = dict(
                            srctype  = 'snowflake',
                            server   = snow_creds['server'],     
                            userName = snow_creds['userName'],
                            password = snow_creds['password'],
                            database = "SNOWFLAKE_SAMPLE_DATA", 
                            schema   = "TPCH_SF10", 
                       ))

View available tables in Snowflake.

In [17]:
conn.fileInfo(caslib = 'my_snow_db')

,Catalog,Schema,Name,Type,Description
0,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,CUSTOMER,TABLE,Customer data as defined by TPC-H
1,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,LINEITEM,TABLE,Lineitem data as defined by TPC-H
2,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,NATION,TABLE,Nation data as defined by TPC-H
3,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,ORDERS,TABLE,Orders data as defined by TPC-H
4,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,PART,TABLE,Part data as defined by TPC-H
5,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,PARTSUPP,TABLE,Partsupp data as defined by TPC-H
6,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,REGION,TABLE,Region data as defined by TPC-H
7,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,SUPPLIER,TABLE,Supplier data as defined by TPC-H


View available in-memory CAS tables in the **my_snow_db** caslib (should be none since we haven't loaded anything into memory on the CAS server).

In [18]:
conn.tableInfo(caslib = 'my_snow_db')

NOTE: No tables are available in caslib my_snow_db of Cloud Analytic Services.


+ Elapsed: 0.00712s, user: 0.00576s, sys: 0.00518s, mem: 1.45mb

## Executing SQL Queries

Load the [fedSQL action set](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg/cas-fedsql-TblOfActions.htm?fromDefault=).

In [7]:
conn.loadActionSet('fedSQL')

NOTE: Added action set 'fedSQL'.


[actionset]

 'fedSQL'

+ Elapsed: 0.0101s, user: 0.00305s, sys: 0.0147s, mem: 1.21mb

In [8]:
conn.fedSQL?

Signature:   conn.fedSQL(*args, **kwargs)
Type:        Fedsql
String form: <swat.cas.actions.Fedsql object at 0x000001EAF1932890>
File:        c:\users\pestyl\appdata\local\anaconda3\lib\site-packages\swat\cas\actions.py
Docstring:  
FedSQL

Actions
-------
fedsql.execdirect : Submits a SAS FedSQL language statement for immediate
                    execution

### Use SAS FedSQL

Execute a simple SQL query through the execDirect action. The method option prints a brief description of the FedSQL query plan. Notice that SAS will pass the ANSI SQL query to Snowflake for execution when possible.

In [21]:
myQuery = '''
    SELECT P_MFGR, count(*)
    FROM MY_SNOW_DB.PART
    GROUP BY P_MFGR
'''
conn.execDirect(query = myQuery, method = True)

 
Methods for full query plan
----------------------------
        Agg 
          Sort 
            SeqScan from my_snow_db.PART 
 
Offloaded SQL statement
------------------------
 
        select "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF10"."PART"."P_MFGR", COUNT ( * )  as "COUNT" from "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF10"."PART" group by "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF10"."PART"."P_MFGR"
 
NOTE: The SQL statement was fully offloaded to the underlying data source via full pass-through


,P_MFGR,COUNT
0,Manufacturer#4,400154.0
1,Manufacturer#1,399506.0
2,Manufacturer#2,399091.0
3,Manufacturer#3,400964.0
4,Manufacturer#5,400285.0


In [19]:
myQuery = '''
    SELECT COUNT(*) as TotalRows
    FROM MY_SNOW_DB.PART
'''
conn.execDirect(query = myQuery, method = True)

 
Methods for full query plan
----------------------------
        Agg 
          SeqScan from my_snow_db.PART 
 
Offloaded SQL statement
------------------------
 
        select COUNT ( * )  as "TOTALROWS" from "SNOWFLAKE_SAMPLE_DATA"."TPCH_SF10"."PART"
 
NOTE: The SQL statement was fully offloaded to the underlying data source via full pass-through


,TOTALROWS
0,2000000.0


You can disable SAS implicit pass through. If you disable pass through and the data is not loaded into memory on the CAS server, a serial load is performed. The table is only loaded into memory for the query and then offloaded. You can load the table for additional processing if you choose to.

In [10]:
myQuery = '''
    SELECT COUNT(*) as TotalRows
    FROM MY_SNOW_DB.PART
'''
conn.execDirect(query = myQuery, method = True, cntl={'disablePassThrough':True})

 
Methods for full query plan
----------------------------
        Agg 
          SeqScan from my_snow_db.PART 
 
NOTE: Performing serial LoadTable action using SAS Data Connector to Snowflake.
Methods for stage 1
--------------------
        No action required
 
Methods for stage 2
--------------------
        Parallel aggregation with the aggregation.aggregate action will be performed
 
        Input table: MY_SNOW_DB.PART
        Output columns:
          ntotal(__fedsql_star__) as __expr_1
        Computed vars program: __fedsql_star__ = 1; 
 


,TOTALROWS
0,2000000


View the available CAS tables in the caslib. Notice that SAS temporarily loaded the table into CAS to process the query (load-on-demand capability). Most of the time CAS uses serial-load for temporary tables, which can impact the SQL performance. The table is also loaded temporarily, so you can't process the CAS table again.

In [12]:
conn.tableInfo(caslib = 'my_snow_db')

NOTE: No tables are available in caslib my_snow_db of Cloud Analytic Services.


+ Elapsed: 0.00594s, user: 0.00379s, sys: 0.00557s, mem: 1.46mb

Execute another fedSQL query. Notice that again SAS temporarily loads the table into memory to process the query. Moving data from Snowflake back to SAS again, taking another 30 seconds to execute.

In [13]:
myQuery = '''
    SELECT *
    FROM MY_SNOW_DB.PART
    LIMIT 10
'''

conn.execDirect(query = myQuery, method = True)

 
Methods for full query plan
----------------------------
        Limit 
          SeqScan from my_snow_db.PART 
 
NOTE: Performing serial LoadTable action using SAS Data Connector to Snowflake.
Methods for stage 1
--------------------
        Limit 
          SeqScan from my_snow_db.PART 
 


,P_PARTKEY,P_NAME,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RETAILPRICE,P_COMMENT
0,260004.0,cornflower seashell brown cornsilk coral,Manufacturer#2,Brand#21,STANDARD BRUSHED NICKEL,12.0,LG BOX,963.99,deposits bo
1,260008.0,rosy smoke dim grey tan,Manufacturer#5,Brand#55,LARGE PLATED STEEL,33.0,JUMBO CASE,967.99,express d
2,260012.0,rosy saddle azure tomato blue,Manufacturer#3,Brand#34,ECONOMY PLATED STEEL,29.0,LG PACK,972.00,"nusual,"
3,260016.0,grey gainsboro coral forest green,Manufacturer#4,Brand#45,PROMO PLATED NICKEL,14.0,JUMBO CASE,976.00,fluffily unusual pin
4,260020.0,metallic bisque dark chiffon tomato,Manufacturer#5,Brand#52,SMALL ANODIZED BRASS,13.0,WRAP JAR,980.01,s cajo
5,260024.0,bisque navajo azure tomato lime,Manufacturer#5,Brand#51,LARGE ANODIZED BRASS,21.0,MED BAG,984.01,the even deposits. fi
6,260028.0,tomato lavender midnight peru plum,Manufacturer#1,Brand#12,ECONOMY ANODIZED BRASS,27.0,LG CAN,988.01,s. ironi
7,260032.0,cornflower tomato blanched frosted mint,Manufacturer#2,Brand#22,LARGE PLATED STEEL,49.0,LG PKG,992.02,against
8,260036.0,forest antique deep cyan red,Manufacturer#2,Brand#23,ECONOMY PLATED STEEL,1.0,MED JAR,996.02,ily among the careful
9,260040.0,plum blue thistle orchid red,Manufacturer#3,Brand#32,STANDARD POLISHED TIN,44.0,WRAP PACK,1000.03,regular foxes over


### Use Snowflake SQL to process in the database

Instead of using a temporary load, you can use SAS explicit pass through to push all of the processing into Snowflake, and only returning the smaller results back.

I'll first run the fileInfo action again to view the available tables in the database. You will see the actual database table name (SNOWFLAKE_SAMPLE_DATA), the schema (TPCH_SF10), and all of the available tables.

In [24]:
conn.fileInfo(caslib = 'my_snow_db')

,Catalog,Schema,Name,Type,Description
0,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,CUSTOMER,TABLE,Customer data as defined by TPC-H
1,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,LINEITEM,TABLE,Lineitem data as defined by TPC-H
2,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,NATION,TABLE,Nation data as defined by TPC-H
3,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,ORDERS,TABLE,Orders data as defined by TPC-H
4,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,PART,TABLE,Part data as defined by TPC-H
5,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,PARTSUPP,TABLE,Partsupp data as defined by TPC-H
6,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,REGION,TABLE,Region data as defined by TPC-H
7,SNOWFLAKE_SAMPLE_DATA,TPCH_SF10,SUPPLIER,TABLE,Supplier data as defined by TPC-H


I'll use implicit pass through to run a native Snowflake query. This will process the query directly in Snowflake and return only the results back.

In the Snowflake query I'll use the Snowflake specific keyword EXCLUDE to speicify the columns that should be excluded from the results. EXCLUDE is not a SAS fedSQL keyword. Notice that this query ran in about 1 second since the processing occurred in Snowflake.

For more information, check out [FedSQL Explicit Pass-Through Facility](https://go.documentation.sas.com/doc/en/pgmsascdc/default/casfedsql/p15c6bokfxufuen1s1by4v3zljq7.htm#p1lmzyifgmgk1pn1l337com6jwxd)

In [9]:
myQuery = '''
    SELECT * FROM CONNECTION TO MY_SNOW_DB
        (SELECT * exclude(P_COMMENT, P_NAME)
         FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.PART
         LIMIT 10)
'''

myQuery = '''
    SELECT * FROM CONNECTION TO MY_SNOW_DB
        (SELECT * exclude(P_COMMENT, P_NAME)
         FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.PART
         LIMIT 10)
'''

conn.execDirect(query = myQuery, 
                method = True)

 
Methods for full query plan
----------------------------
        SeqScan from my_snow_db.__fedsql_cep_1__ 
 
Methods for stage 1
--------------------
        FedSQL did not generate a plan. Entire query can be pushed to driver.
 


,P_PARTKEY,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RETAILPRICE
0,540001.0,Manufacturer#1,Brand#12,SMALL BRUSHED BRASS,42.0,SM DRUM,1040.98
1,540002.0,Manufacturer#5,Brand#55,SMALL POLISHED BRASS,43.0,SM PKG,1041.98
2,540003.0,Manufacturer#1,Brand#11,ECONOMY PLATED BRASS,31.0,JUMBO JAR,1042.98
3,540004.0,Manufacturer#1,Brand#11,LARGE BURNISHED BRASS,40.0,LG BAG,1043.98
4,540005.0,Manufacturer#5,Brand#54,ECONOMY ANODIZED TIN,21.0,JUMBO BAG,1044.98
5,540006.0,Manufacturer#1,Brand#13,LARGE BURNISHED STEEL,10.0,SM PKG,1045.98
6,540007.0,Manufacturer#4,Brand#44,STANDARD POLISHED TIN,48.0,JUMBO JAR,1046.98
7,540008.0,Manufacturer#3,Brand#32,STANDARD POLISHED TIN,10.0,MED BOX,1047.98
8,540009.0,Manufacturer#5,Brand#53,ECONOMY POLISHED BRASS,6.0,LG PACK,1048.98
9,540010.0,Manufacturer#5,Brand#52,LARGE BURNISHED NICKEL,49.0,SM PKG,1049.99


## Loading Snowflake data to the CAS server

Load the Snowflake table into memory in the CAS server. Once the data is loaded in CAS you can use available Pandas API methods or CAS actions from the SWAT package to process your data. Loading it into memory avoids the temporary loading we saw earlier.

Here I'll load the Snowflake table into memory in the **Casuser** caslib and name the CAS table **PART_CAS_TABLE**.

Work with the results I got from Snowflake and load to CAS and give me the smaller results.

In [11]:
myQuery = '''
    CREATE TABLE CASUSER.PART_CAS_TABLE as
    SELECT * FROM CONNECTION TO MY_SNOW_DB
        (SELECT * exclude(P_COMMENT, P_NAME)
         FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.PART);
'''

conn.execDirect(query = myQuery, 
                method = True)

 
Methods for full query plan
----------------------------
        SeqScan from my_snow_db.__fedsql_cep_1__ 
 
Methods for stage 1
--------------------
        SeqScan from {Push Down}.Child 1 
 
NOTE: Table PART_CAS_TABLE was created in caslib CASUSER(Peter.Styliadis@sas.com) with 2000000 rows returned.


+ Elapsed: 4.45s, user: 5.44s, sys: 3.43s, mem: 521mb

View available in-memory CAS tables.

In [12]:
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,PART_CAS_TABLE,2000000,7,0,utf-8,2023-08-18T11:37:18+00:00,2023-08-18T11:37:18+00:00,2023-08-18T11:37:18+00:00,UTF8,2.007978e+09,2.007978e+09,2.007978e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
1,GOV_IT_BUDGET_NARROW,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-16T15:15:37+00:00,UTF8,2.007817e+09,2.007817e+09,2.007818e+09,1,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09


Reference the CAS table on the client.

In [13]:
castbl = conn.CASTable('PART_CAS_TABLE', caslib = 'casuser')

Preview the CAS table.

In [14]:
castbl.head()

,P_PARTKEY,P_MFGR,P_BRAND,P_TYPE,P_SIZE,P_CONTAINER,P_RETAILPRICE
0,1458433.0,Manufacturer#5,Brand#52,ECONOMY PLATED NICKEL,37.0,WRAP JAR,1391.36
1,1458434.0,Manufacturer#1,Brand#14,SMALL PLATED TIN,49.0,SM BOX,1392.36
2,1458435.0,Manufacturer#4,Brand#45,STANDARD BRUSHED BRASS,48.0,MED PKG,1393.36
3,1458436.0,Manufacturer#4,Brand#43,LARGE BURNISHED BRASS,7.0,MED CASE,1394.36
4,1458437.0,Manufacturer#2,Brand#23,SMALL ANODIZED BRASS,12.0,MED BOX,1395.36


In [16]:
castbl.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,P_PARTKEY,,1,double,8,12,,0,0
1,P_MFGR,,2,varchar,25,25,,0,0
2,P_BRAND,,3,varchar,10,10,,0,0
3,P_TYPE,,4,varchar,25,25,,0,0
4,P_SIZE,,5,double,8,12,,0,0
5,P_CONTAINER,,6,varchar,10,10,,0,0
6,P_RETAILPRICE,,7,double,8,12,,0,0


In [17]:
castbl.nmiss()

P_PARTKEY        0
P_MFGR           0
P_BRAND          0
P_TYPE           0
P_SIZE           0
P_CONTAINER      0
P_RETAILPRICE    0
dtype: int64

In [18]:
castbl.summary()

,Column,Min,Max,N,NMiss,Mean,Sum,Std,StdErr,Var,USS,CSS,CV,TValue,ProbT,Skewness,Kurtosis
0,P_PARTKEY,1.00,2000000.00,2000000.0,0.0,1.000000e+06,2.000001e+12,577350.413527,408.248393,3.333335e+11,2.666669e+18,6.666667e+17,57.735012,2449.490355,0.0,3.373649e-14,-1.200000
1,P_SIZE,1.00,50.00,2000000.0,0.0,2.551474e+01,5.102949e+07,14.425957,0.010201,2.081082e+02,1.718221e+09,4.162163e+08,56.539692,2501.275692,0.0,-1.762660e-03,-1.200247
2,P_RETAILPRICE,900.91,2098.99,2000000.0,0.0,1.499495e+03,2.998990e+09,294.603166,0.208316,8.679103e+04,4.670553e+12,1.735820e+11,19.646824,7198.178966,0.0,7.219406e-04,-1.112162


In [19]:
## Load the freqTab action
conn.loadActionSet('freqTab')

## Specify the columns to analyze
catColumns = ['P_MFGR','P_BRAND','P_CONTAINER']

## excute the freqTab action
castbl.freqTab(tabulate = catColumns)

NOTE: Added action set 'freqTab'.


[LevelInfo]

 Variable Level Information
 
       Variable  Levels                                                                                                                                                                                                                                                                                                                                                  Values
 0      P_BRAND      25                                                                                                                        Brand#11 Brand#12 Brand#13 Brand#14 Brand#15 Brand#21 Brand#22 Brand#23 Brand#24 Brand#25 Brand#31 Brand#32 Brand#33 Brand#34 Brand#35 Brand#41 Brand#42 Brand#43 Brand#44 Brand#45 Brand#51 Brand#52 Brand#53 Brand#54 Brand#55
 1  P_CONTAINER      40  JUMBO BAG JUMBO BOX JUMBO CAN JUMBO CASE JUMBO DRUM JUMBO JAR JUMBO PACK JUMBO PKG LG BAG LG BOX LG CAN LG CASE LG DRUM LG JAR LG PACK LG PKG MED BAG MED BOX MED CAN MED CASE MED DRUM MED JAR MED PACK MED PKG SM BAG SM BOX SM CAN SM CASE SM DRUM SM JAR SM PACK SM PKG WRAP BAG WRAP BOX WRAP CAN WRAP CASE WRAP DRUM WRAP JAR WRAP PACK WRAP PKG
 2       P_MFGR       5                                                                                                                                                                                                                                                                              Manufacturer#1 Manufacturer#2 Manufacturer#3 Manufacturer#4 Manufacturer#5

[NObs]

 Number of Observations
 
       RowId                  Description      Value
 0     NREAD  Number of Observations Read  2000000.0
 1     NUSED  Number of Observations Used  2000000.0
 2  NWEIGHTS               Sum of Weights  2000000.0

[Table1.OneWayFreqs]

 P_MFGR
 
          F_P_MFGR          P_MFGR  Frequency   Percent  CumFrequency  CumPercent
 0  Manufacturer#1  Manufacturer#1   399506.0  19.97530      399506.0    19.97530
 1  Manufacturer#2  Manufacturer#2   399091.0  19.95455      798597.0    39.92985
 2  Manufacturer#3  Manufacturer#3   400964.0  20.04820     1199561.0    59.97805
 3  Manufacturer#4  Manufacturer#4   400154.0  20.00770     1599715.0    79.98575
 4  Manufacturer#5  Manufacturer#5   400285.0  20.01425     2000000.0   100.00000

[Table2.OneWayFreqs]

 P_BRAND
 
    F_P_BRAND   P_BRAND  Frequency  Percent  CumFrequency  CumPercent
 0   Brand#11  Brand#11    79150.0  3.95750       79150.0     3.95750
 1   Brand#12  Brand#12    79980.0  3.99900      159130.0     7.95650
 2   Brand#13  Brand#13    80013.0  4.00065      239143.0    11.95715
 3   Brand#14  Brand#14    80404.0  4.02020      319547.0    15.97735
 4   Brand#15  Brand#15    79959.0  3.99795      399506.0    19.97530
 5   Brand#21  Brand#21    80094.0  4.00470      479600.0    23.98000
 6   Brand#22  Brand#22    79669.0  3.98345      559269.0    27.96345
 7   Brand#23  Brand#23    79826.0  3.99130      639095.0    31.95475
 8   Brand#24  Brand#24    79781.0  3.98905      718876.0    35.94380
 9   Brand#25  Brand#25    79721.0  3.98605      798597.0    39.92985
 10  Brand#31  Brand#31    80015.0  4.00075      878612.0    43.93060
 11  Brand#32  Brand#32    79812.0  3.99060      958424.0    47.92120
 12  Brand#33  Brand#33    80508.0  4.02540     1038932.0    51.94660
 13  Brand#34  Brand#34    80043.0  4.00215     1118975.0    55.94875
 14  Brand#35  Brand#35    80586.0  4.02930     1199561.0    59.97805
 15  Brand#41  Brand#41    79918.0  3.99590     1279479.0    63.97395
 16  Brand#42  Brand#42    79652.0  3.98260     1359131.0    67.95655
 17  Brand#43  Brand#43    80170.0  4.00850     1439301.0    71.96505
 18  Brand#44  Brand#44    80288.0  4.01440     1519589.0    75.97945
 19  Brand#45  Brand#45    80126.0  4.00630     1599715.0    79.98575
 20  Brand#51  Brand#51    80123.0  4.00615     1679838.0    83.99190
 21  Brand#52  Brand#52    80260.0  4.01300     1760098.0    88.00490
 22  Brand#53  Brand#53    80106.0  4.00530     1840204.0    92.01020

## Promote the table to use Visual Analytics to create a dashboard

In [27]:
castbl.table.promote(targetLib = 'casuser')

NOTE: Cloud Analytic Services promoted table PART_CAS_TABLE in caslib CASUSER(Peter.Styliadis@sas.com) to table PART_CAS_TABLE in caslib CASUSER(Peter.Styliadis@sas.com).


+ Elapsed: 0.0224s, user: 0.00934s, sys: 0.0197s, mem: 1.39mb

Image of VA

## Terminate the CAS session

In [9]:
conn.terminate()

# Additional Resources

- [Getting Started with Python Integration to SAS® Viya® - Index](https://blogs.sas.com/content/sgf/2020/06/19/getting-started-with-python-integration-to-sas-viya-index/)
- [FedSQL Action Set: Syntax](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg/cas-fedsql-TblOfActions.htm?fromDefault=)
- [SAS Viya Best practices with Snowflake Data](https://video.sas.com/detail/video/6312274491112/sas-viya-best-practices-with-snowflake-data)
- [Documentation - Snowflake Data Connector](https://go.documentation.sas.com/doc/en/pgmsascdc/default/casref/p183rli8obtde3n10y9bzbrpwnsh.htm)
- [FedSQL Implicit Pass-Through Facility for CAS](https://communities.sas.com/t5/SAS-Communities-Library/FedSQL-Implicit-Pass-Through-Facility-for-CAS/ta-p/459556)
- [Proc FedSQL and Multi-node load to CAS](https://communities.sas.com/t5/SAS-Communities-Library/Proc-FedSQL-and-Multi-node-load-to-CAS/ta-p/824773)